## Ingestion to PostgreSQL

#### Set environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

pg_host  = os.getenv("POSTGRESQL_HOST")
if pg_host is None or pg_host == "":
    print("POSTGRESQL_HOST environment variable not set.")
    exit()

pg_user  = os.getenv("POSTGRESQL_USERNAME")
if pg_user is None or pg_user == "":
    print("POSTGRESQL_USERNAME environment variable not set.")
    exit()

pg_password  = os.getenv("POSTGRESQL_PASSWORD")
if pg_password is None or pg_password == "":
    print("POSTGRESQL_PASSWORD environment variable not set.")
    exit()

db_name  = os.getenv("POSTGRESQL_DATABASE")
if db_name is None or db_name == "":
    print("POSTGRESQL_DATABASE environment variable not set.")
    exit()

text_table_name = 'text_sample'
doc_table_name = 'doc_sample'
image_table_name = 'image_sample'

postgresql_params = {
    "host": pg_host,
    "port": "5432", 
    "dbname": db_name,
    "user": pg_user,
    "password": pg_password
}

#### Add vector extension

The vector extension needs to be enabled in every database.

In [ ]:
from psycopg2 import connect

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        cursor.execute('CREATE EXTENSION IF NOT EXISTS vector;')
        
        print('Vector extension added.')

#### Create table

In [ ]:
from psycopg2 import connect

def create_table(table_name, table_schema):

    with connect(**postgresql_params) as connection:
        with connection.cursor() as cursor:
            cursor.execute(f"DROP TABLE IF  EXISTS {table_name} ")
            cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({table_schema});")

            print(f"Table {table_name} created.")

## Create text_sample table
table_schema = """
    id smallint PRIMARY KEY,
    title text,
    content text,
    category text,
    title_vector VECTOR(1536),
    content_vector VECTOR(1536)
 """
create_table(text_table_name, table_schema)

## Create doc_sample table
table_schema = """
    id smallint PRIMARY KEY,
    chunk_content text,
    chunk_content_vector VECTOR(1536)
 """
create_table(doc_table_name, table_schema)

## Create image_sample table
table_schema = """
    id smallint PRIMARY KEY,
    image text,
    image_vector VECTOR(1024)
 """
create_table(image_table_name, table_schema)

#### Ingest text sample with embeddings

In [ ]:
import pandas as pd

text_df = pd.read_json('../data/text/product_docs_embeddings.json')
records = text_df.values.tolist()

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        insert_sql = f"INSERT INTO {text_table_name}(id, title, content, category, title_vector, content_vector) VALUES(%s, %s, %s, %s, %s, %s)"
        cursor.executemany(insert_sql, records)

        print("Text sample ingested.")

#### Ingest doc sample with embeddings

In [ ]:
import pandas as pd

doc_df = pd.read_json('../data/docs/employee_handbook_embeddings.json')
records = doc_df.values.tolist()

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        insert_sql = f"INSERT INTO {doc_table_name}(id, chunk_content, chunk_content_vector) VALUES(%s, %s, %s)"
        cursor.executemany(insert_sql, records)

        print("Doc sample ingested.")

#### Ingest image sample with embeddings

In [ ]:
import pandas as pd

image_df = pd.read_json('../data/images/images_embeddings.json')
records = image_df.values.tolist()

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        insert_sql = f"INSERT INTO {image_table_name}(id, image, image_vector) VALUES(%s, %s, %s)"
        cursor.executemany(insert_sql, records)

        print("Image sample ingested.")

#### Create IVFFlat Index

An IVFFlat index divides vectors into lists, and then searches a subset of those lists that are closest to the query vector. Details - https://github.com/pgvector/pgvector#ivfflat

In [ ]:
import math

def get_query_suffix(row_count):
    query_suffix = ''
    if row_count >= 1000:
        # Determine the number of lists based on the number of rows
        if row_count <= 1000000:
            lists = row_count / 1000
        else:
            lists = math.sqrt(row_count)
        query_suffix = f"WITH (lists = {int(lists)})"  

    return query_suffix

# Create index for text_sample table 

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        cursor.execute(f'SELECT count(*) FROM {text_table_name}')
        row_count = cursor.fetchone()[0]

        query_suffix = get_query_suffix(row_count)

        index_query = f"""
            CREATE INDEX IF NOT EXISTS ix_title_vector_cosine ON {text_table_name} USING ivfflat (title_vector vector_cosine_ops) {query_suffix};
            CREATE INDEX IF NOT EXISTS ix_content_vector_cosine ON {text_table_name} USING ivfflat (content_vector vector_cosine_ops) {query_suffix};
        """
        cursor.execute(index_query)

        print(f"IVFFlat index created for {text_table_name}.")

# Create index for doc_sample table 

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        cursor.execute(f'SELECT count(*) FROM {doc_table_name}')
        row_count = cursor.fetchone()[0]

        query_suffix = get_query_suffix(row_count)

        index_query = f"""
            CREATE INDEX IF NOT EXISTS ix_chunk_content_vector_cosine ON {doc_table_name} USING ivfflat (chunk_content_vector vector_cosine_ops) {query_suffix};
        """
        cursor.execute(index_query)

        print(f"IVFFlat index created for {doc_table_name}.")

# Create index for image_sample table 

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        cursor.execute(f'SELECT count(*) FROM {image_table_name}')
        row_count = cursor.fetchone()[0]

        query_suffix = get_query_suffix(row_count)

        index_query = f"""
            CREATE INDEX IF NOT EXISTS ix_image_vector_cosine ON {image_table_name} USING ivfflat (image_vector vector_cosine_ops) {query_suffix};
        """
        cursor.execute(index_query)

        print(f"IVFFlat index created for {image_table_name}.")